<h1>Roof & terrace modeling</h1>

<h2>Reference models</h2>
<h3>Simple:</h3>
![alt text](reference-model/model.jpg)
<h3>Complex:</h3>
![alt text](reference-model/model2.jpg)

<h4>Description of the geometric methods i'm going to implement:</h4>
The primary method for the creation of the roof from a floor plan, takes as input 3 parameters, representing .lines file, the angle of inclination and length of the slope. This method has various methods to support: <b>vertexFlap</b> allows me to return to the 4 vertexes of the plane belonging to the flap, necessary to define the various flap; <b>points2line</b> that returns a line from two points, necessary to <b>linesIntersection</b> method that two straight dates me from the point of intersection; <b>getdirections</b> returns me the "direction" of all plans Belonging to all flaps; <b>getVertexes</b> return all vertexes of .lines files as input that represents the floor plan of the house. The main method starts with defining the vertexes of the roof from the floor plan and the direction of the flaps. The flaps will then be directed in relation to the adjacent flaps. Then I define the various lines and their points of intersection, and then make the final flaps with appropriate direction and texture. Then I define the border vertexes that allow me to define the edges of the terrace I create with POLYLINE and solidify; The terrace is appropriately translated and completed with the texture, to structure it along to the roof created earlier.

Import libraries

In [1]:
from pyplasm import *
from scipy import *  
from numpy import *  
import math
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.00597415150814 seconds


Define support method to return the 4 vertexes of the plan belonging to the flap

In [2]:
"""input: lists of the coordinates xyz of the two vertex (vertex1,vertex2),
   the inclination angle of the flaps (angle),
   flap's length (length)
   direction's quadrant of the flap (direction)
   output: all the flap's vertexes"""
def vertexFlap(vertex1, vertex2, angle, length, direction):

    #order x & y
    if vertex1[1]>vertex2[1]:
        x=vertex1[0]
        y=vertex2[1]
    else:
        x=vertex2[0]
        y=vertex1[1]

    vertex3 = [x,y,0]

    #AB = sqrt[(x2- x1)^2 + (y2- y1)^2]
    distance12 = sqrt((vertex1[0]-vertex2[0])*(vertex1[0]-vertex2[0])
                      +(vertex1[1]-vertex2[1])*(vertex1[1]-vertex2[1]))
    distance13 = sqrt((vertex1[0]-vertex3[0])*(vertex1[0]-vertex3[0])
                      +(vertex1[1]-vertex3[1])*(vertex1[1]-vertex3[1]))
    distance23 = sqrt((vertex2[0]-vertex3[0])*(vertex2[0]-vertex3[0])
                      +(vertex2[1]-vertex3[1])*(vertex2[1]-vertex3[1]))
    
    #distance23 = distance12 * cos(a)
    #cos(a) = distance23/distance12
    a = math.asin(distance23/distance12)
    b = PI/2-a

    distance24 = length * math.cos(angle)

    h = sqrt(length*length-distance24*distance24)
    distance25 = distance24 * math.cos(b)
    distance45 = sqrt(distance24*distance24-distance25*distance25)

    #verify the quadrant's number
    if direction==1:
        vertex6 = [vertex2[0]+distance25,vertex2[1]+distance45,h]
        vertex7 = [vertex1[0]+distance25,vertex1[1]+distance45,h]
    elif direction==2:
        vertex6 = [vertex2[0]-distance25,vertex2[1]+distance45,h]
        vertex7 = [vertex1[0]-distance25,vertex1[1]+distance45,h]
    elif direction==3:
        vertex6 = [vertex2[0]-distance25,vertex2[1]-distance45,h]
        vertex7 = [vertex1[0]-distance25,vertex1[1]-distance45,h]
    elif direction==4:
        vertex6 = [vertex2[0]+distance25,vertex2[1]-distance45,h]
        vertex7 = [vertex1[0]+distance25,vertex1[1]-distance45,h]

    #add all vertex find
    allVertexes = [vertex1,vertex2,vertex6,vertex7]

    return allVertexes

Define support method to return a line, starting from 2 points

In [3]:
"""input: 2 lists of coordinates xy of the 2 vertexes/points
   output: a line as list of 3 elements (x,y,n, where n is the sum of x,y)"""
def points2line(vertex1,vertex2):

    x1=vertex1[0]
    x2=vertex2[0]
    y1=vertex1[1]
    y2=vertex2[1]
    m=0
    q=0

    if x1==x2:            #if the vertexes have same x, the line is parallel to y axis
        line = [1,0,x1]
    elif y1==y2:          #if the vertexes have same y, the line is parallel to x axis
        line = [0,1,y1]
    else:
        m=(float(y2)-float(y1))/(float(x2)-float(x1))
        q=float(y1)-m*float(x1)
        line = [-m,1,q]

    return line

Define support method to return the intersection's point betwen 2 lines

In [4]:
"""input: 2 lines
    output: intersection's point"""
def linesIntersection(line1,line2):
      
    #coefficients on the left of the equals
    A = matrix([[line1[0], line1[1]], [line2[0], line2[1]]])  
    
    #knowns terms 
    b = array([line1[2], line2[2]])  
      
    #resolve linear system
    intersectionPoint = linalg.solve(A, b)
    
    return intersectionPoint

Define support method to determinate the directions of all plan belonging to all flaps

In [5]:
"""input: list of vertexes of all plans belonging to all flaps
   output: list of the directions of all flaps"""
def getDirections(vertexes):
    
    directions=[]
    
    for i in range(1,len(vertexes)):
        x1=vertexes[i-1][0]
        x2=vertexes[i][0]
        y1=vertexes[i-1][1]
        y2=vertexes[i][1]
        
        #compare the vertexes to determinate the quadrant's number
        if x2>=x1:
            if y2>=y1:
                directions.append(4)
            else:
                directions.append(3)  
        else:
            if y2>=y1:
                directions.append(1)
            else:
                directions.append(2)
                
    return directions

Define support method to get all the vertexes of the input's .lines file

In [6]:
"""input: .lines file
   output: list of all vertexes of the floor plan"""
def getVertexes(path):
    
    with open(path, "rb") as file:
        reader = csv.reader(file, delimiter=",")
        
        #append points as couple of coordinates xy (01 & 23)
        temp=[]
        for row in reader:
            temp.append([float(row[0]),float(row[1])])
            temp.append([float(row[2]),float(row[3])])

        #take only vertexes with even index (because they are the exactly vertexes of the floor plan)
        vertexes=[]
        for i in range(len(temp)):
            if i%2==0:
                vertexes.append(temp[i])
            if i==len(temp)-3:
                vertexes.append(temp[i])
                
    return vertexes

Define main function to create final roof&terrace starting by a .lines file, inclination's angle of flaps, and length of the flaps

In [7]:
"""input: path of .lines's file, angle of flaps's inclinations, flaps's length
   output: final structure of roof & terrace"""
def ggpl_roof_terrace(path,angle,length):

    #define the vertexes of the floor plan, and the directions of all plan of alla flaps
    vertexes=getVertexes(path)
    directions=getDirections(vertexes)

    #opportunely direction of the flaps in relation to the adjacent flaps
    flaps = []
    for i in range(len(directions)):
        if i==len(directions)-1:
            flaps.append(vertexFlap(vertexes[i],vertexes[0],angle,length,directions[i]))
        else:
            s = i+1
            flaps.append(vertexFlap(vertexes[i],vertexes[s],angle,length,directions[i]))

    #define lines and their intersection's point, of all lines in the "roof"
    lines = []
    for i in range(len(flaps)):
        lines.append(points2line(flaps[i][2],flaps[i][3]))

    intersections = []
    for i in range(len(lines)):
        if i==len(lines)-1:
            intersections.append(linesIntersection(lines[i],lines[0]))
            intersections.append(linesIntersection(lines[i],lines[0]))
        else:
            s = i+1
            intersections.append(linesIntersection(lines[i],lines[s]))

    #make final flaps with opportune direction and texture
    finalFlaps = []
    for i in range(len(directions)):
        
        if i == 0:           
            f = MKPOL([[
                        [flaps[i][0][0],flaps[i][0][1],0],
                        [flaps[i][1][0],flaps[i][1][1],0],
                        [intersections[i][0],intersections[i][1],flaps[1][2][2]],
                        [intersections[len(directions)-1][0],intersections[len(directions)-1][1],flaps[0][2][2]]],
                       [[1,2,3,4]],None])
            
            f = TEXTURE(["texture/roof.jpg", TRUE, FALSE, 1, 1, 0, 10, 10])(f)            
            finalFlaps.append(f)
            
        else:
            f = MKPOL([[[flaps[i][0][0],flaps[i][0][1],0],
                        [flaps[i][1][0],flaps[i][1][1],0],
                        [intersections[i][0],intersections[i][1],flaps[1][2][2]],
                        [intersections[i-1][0],intersections[i-1][1],flaps[0][2][2]]],
                       [[1,2,3,4]],None])
            
            f = TEXTURE("texture/roof.jpg")(f)
            finalFlaps.append(f)

    #define border vertexes for create the terrace as polyline+solidify
    borderVertexes = [[] for _ in range(len(intersections)+1)]
    cell = []
    for i in range (len(intersections)):
        borderVertexes[i].append(intersections[i][0])
        borderVertexes[i].append(intersections[i][1])
        s = i+1
        if i==len(intersections)-1:
            borderVertexes[s].append(intersections[0][0])
            borderVertexes[s].append(intersections[0][1])
        cell.append(s)

    border = POLYLINE(borderVertexes)
    border = SOLIDIFY(border)

    #opportunely shift and texture
    terrace = T(3)(flaps[0][2][2])(border)
    terrace = TEXTURE(["texture/terrace.jpg", TRUE, FALSE, 1, 1, 0, 10, 10])(terrace)
    
    #struct final roof and roof+terrace
    roof = STRUCT(finalFlaps)
    roofTerrace=STRUCT([terrace,roof])
    
    return roofTerrace

Launch function

In [8]:
roof_terrace = ggpl_roof_terrace("perimeter2.lines",PI/3,80)
VIEW(roof_terrace)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000AFFB480> >

<h2>Final result!</h2>
<h3>Simple -> Angle: Pi/4, length: 180</h3>
![alt text](screenshots/1.jpg)
![alt text](screenshots/2.jpg)
![alt text](screenshots/3.jpg)
![alt text](screenshots/4.jpg)
<h3>Complex -> Angle: Pi/3, length: 80</h3>
![alt text](screenshots/5.jpg)
![alt text](screenshots/6.jpg)
![alt text](screenshots/7.jpg)
![alt text](screenshots/8.jpg)